### Обогащаем выборку геоданными и анализируем текущие данные

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('realty_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98822 entries, 0 to 98821
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  98822 non-null  object 
 1   period        0 non-null      float64
 2   price         98822 non-null  int64  
 3   postcode      93675 non-null  float64
 4   address_name  98821 non-null  object 
 5   lat           98822 non-null  float64
 6   lon           98822 non-null  float64
 7   object_type   98822 non-null  object 
 8   total_square  98822 non-null  float64
 9   rooms         94840 non-null  float64
 10  floor         98822 non-null  float64
 11  city          91928 non-null  object 
 12  settlement    6894 non-null   object 
 13  district      75111 non-null  object 
 14  area          19498 non-null  object 
 15  description   98573 non-null  object 
 16  source        98822 non-null  object 
dtypes: float64(7), int64(1), object(9)
memory usage: 12.8+ MB


Использование геоданных позволяет более точно определить характеристики объектов недвижимости и их влияние на цену. Одним из способов получения геоданных является использование данных OpenStreetMap (OSM), которые содержат информацию о дорогах, зданиях, общественном транспорте и других объектах. 

С помощью библиотеки OSMnx мы можем загрузить данные OSM и использовать их для получения дополнительной информации о расположении объекта недвижимости. Это позволит создать новые признаки, такие как расстояние до ближайшей остановки общественного транспорта или парка от объекта недвижимости. Эти признаки помогут улучшить качество модели и сделать ее более интерпретируемой.

#### Источники. 

**OpenStreetMap (OSM)** — это проект по созданию свободной, редактируемой карты мира. Карта создается сообществом пользователей, которые вносят информацию о дорогах, зданиях, местах отдыха и других объектах в различных городах и странах. Все данные в OSM доступны для свободного использования и редактирования, что позволяет создавать на их основе новые карты и приложения. OSM является альтернативой коммерческим картам, таким как Google Maps или Yandex Maps.

OSM может быть полезна для прогнозирования цен на недвижимость, так как она содержит информацию о транспортной инфраструктуре, расположении объектов инфраструктуры (школ, магазинов, парков и т.д.), а также о характеристиках зданий и их окружения (высота здания, количество этажей, наличие парковок и т.д.). Эти данные могут помочь определить факторы, влияющие на стоимость недвижимости, и использовать их для прогнозирования будущих цен. Например, наличие хорошей транспортной инфраструктуры и развитой инфраструктуры вокруг здания может повысить его стоимость.

**Работа с OSM**

In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
from folium.plugins import HeatMap
from shapely.geometry import Polygon
from geopy.distance import distance

In [5]:
# Функция принимает на вход геометрические данные и возвращает список координат
def get_lat_lon(geometry):
    # Если геометрические данные являются точкой, то получаем координаты точки и записываем их в переменную coords
    # Если геометрические данные являются полигоном, то получаем координаты центроида полигона и записываем их в переменную coords
    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
    return lat, lon

In [6]:
# функция для получения данных из OSM по заданным тегам и городу
def osm_query(tag, city):
    
    #получаем геометрические данные из OSM по заданным тегам и городу
    gdf = ox.geometries_from_place(city, tag).reset_index()
    
    # добавляем столбец с названием города
    gdf['city'] = np.full(len(gdf), city.split(',')[0])
    
    # добавляем столбец с названием объекта (название тега)
    gdf['object'] = np.full(len(gdf), list(tag.keys())[0])
    
    # добавляем столбец с типом объекта (значение тега)
    gdf['type'] = np.full(len(gdf), tag[list(tag.keys())[0]])
    
    # оставляем только нужные столбцы в датафрейме
    gdf = gdf[['city', 'object', 'type', 'geometry']]
    print(gdf.shape)
    return gdf

[Здесь](https://wiki.openstreetmap.org/wiki/RU:%D0%9E%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D1%8B_%D0%BA%D0%B0%D1%80%D1%82%D1%8B) искала необходимые объекты, которые могут влиять на стоимость жилья: 

In [7]:
# список тегов и городов для получения данных    
tags = [{'building' : 'industrial'}, #промышленное здание
        {'building' : 'office'}, #офисное здание 
        {'building':'train_station'}, #вокзал
        {'highway' : 'bus_stop'}, #остановка
        {'amenity':'school'}, #школьное здание
        {'amenity':'university'}, #ВУЗ, университет 
        {'amenity':'clinic'}, #крупные поликлиники и коммерческие клиники без стационара 
        {'leisure':'park'}, #парки, открытые зелёные площадки для отдыха
        {'leisure':'sports_centre'},#спортивные центры
        {'building':'kindergarten'}, #детский сад
        {'natural':'water'}, #любые водные поверхности - озёра, водоемы и т.д.
       ]

In [8]:
cities = df.city.unique().tolist()

In [9]:
cities = [city + ', Россия' if type(city) == str else city for city in cities]

In [10]:
cities.remove(cities[1])

In [11]:
cities

['Москва, Россия',
 'Балашиха, Россия',
 'Люберцы, Россия',
 'Красногорск, Россия',
 'Химки, Россия',
 'Королёв, Россия',
 'Мытищи, Россия',
 'Пушкино, Россия',
 'Котельники, Россия',
 'Одинцово, Россия',
 'Щёлково, Россия',
 'Дзержинский, Россия',
 'Реутов, Россия',
 'Ивантеевка, Россия',
 'Московский, Россия',
 'Лобня, Россия',
 'Долгопрудный, Россия',
 'Щербинка, Россия',
 'Подольск, Россия',
 'Видное, Россия',
 'Лыткарино, Россия']

In [12]:
# получаем данные по каждому тегу и городу и добавляем в список
gdfs = [osm_query(tag, city) for city in cities for tag in tags]

# объединяем полученные датафреймы в один
infrastruct = pd.concat(gdfs)

# группируем данные по городу, объекту и типу, подсчитываем количество записей и выводим результат
infrastruct.groupby(['city','object','type'], as_index = False).agg({'geometry':'count'})

# получаем координаты/центроиды из геометрических данных
lat, lon = get_lat_lon(infrastruct['geometry'])
infrastruct['lat'] = lat
infrastruct['lon'] = lon

(6073, 4)
(2444, 4)
(212, 4)
(10041, 4)
(1947, 4)
(370, 4)
(1423, 4)
(1174, 4)
(732, 4)
(1893, 4)
(1348, 4)
(1000, 4)
(9, 4)
(1, 4)
(447, 4)
(78, 4)
(6, 4)
(38, 4)
(36, 4)
(40, 4)
(76, 4)
(190, 4)
(76, 4)
(2, 4)
(1, 4)
(189, 4)
(30, 4)
(3, 4)
(23, 4)
(10, 4)
(13, 4)
(35, 4)
(24, 4)
(88, 4)
(29, 4)
(4, 4)
(124, 4)
(25, 4)
(2, 4)
(36, 4)
(21, 4)
(17, 4)
(49, 4)
(47, 4)
(190, 4)
(25, 4)
(2, 4)
(355, 4)
(47, 4)
(6, 4)
(30, 4)
(22, 4)
(28, 4)
(65, 4)
(160, 4)
(300, 4)
(6, 4)
(1, 4)
(130, 4)
(38, 4)
(9, 4)
(21, 4)
(25, 4)
(11, 4)
(19, 4)
(92, 4)
(90, 4)
(6, 4)
(6, 4)
(204, 4)
(39, 4)
(5, 4)
(23, 4)
(32, 4)
(16, 4)
(41, 4)
(29, 4)
(19, 4)
(2, 4)
(5, 4)
(194, 4)
(24, 4)
(11, 4)
(8, 4)
(61, 4)
(2, 4)
(24, 4)
(150, 4)
(15, 4)
(7, 4)
(0, 4)
(52, 4)
(6, 4)
(1, 4)
(7, 4)
(2, 4)
(3, 4)
(8, 4)
(8, 4)
(32, 4)
(2, 4)
(1, 4)
(84, 4)
(21, 4)
(1, 4)
(15, 4)
(10, 4)
(4, 4)
(21, 4)
(36, 4)
(55, 4)
(6, 4)
(3, 4)
(116, 4)
(21, 4)
(0, 4)
(11, 4)
(20, 4)
(5, 4)
(20, 4)
(121, 4)
(9, 4)
(5, 4)
(0, 4)
(51, 4)
(7, 

/var/folders/7l/jh1g9wm112171g1bjt4zytch0000gn/T/ipykernel_27524/373771680.py:5: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
/var/folders/7l/jh1g9wm112171g1bjt4zytch0000gn/T/ipykernel_27524/373771680.py:6: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)


In [13]:
infrastruct['type'].unique().tolist()

['industrial',
 'office',
 'train_station',
 'bus_stop',
 'school',
 'university',
 'clinic',
 'park',
 'sports_centre',
 'kindergarten',
 'water']

Таким образом, получили данные с необходимыми объектами в заданных городах

Найдем координаты этих объектов и вычислим минимальное расстояние квартиры до любого из объектов каждой категории

In [14]:
from scipy.spatial import distance_matrix

In [14]:
objects = infrastruct['type'].unique().tolist()

In [15]:
# Функция для получения координат объектов инфраструктуры и добавления их в датафрейм
def objects_coords(df_зфке):
    objects = infrastruct['type'].unique().tolist()
    for i in range(len(objects)):
        tag = infrastruct[infrastruct['type'] == objects[i]].reset_index()

        # Создаем список координат квартир и объектов данного типа 
        flats_coords = [(lat, lon) for lat, lon in zip(df["lat"], df["lon"])]
        tag_coords = [(lat, lon) for lat, lon in zip(tag["lat"], tag["lon"])]
        
        # Создаем матрицу расстояний между квартирами и объектами данного типа
        dist_matrix = distance_matrix(flats_coords, tag_coords)
        
        # Находим индексы ближайших объектов для каждой квартиры
        nearest_idx = dist_matrix.argmin(axis=1)
        
        # Добавляем столбцы с координатами ближайших объектов данного типа в датафрейм df
        df["lat_" + objects[i]] = tag.loc[nearest_idx, 'lat'].values
        df["lon_" + objects[i]] = tag.loc[nearest_idx, 'lon'].values
    return df

In [ ]:
df = objects_coords(df)

In [ ]:
def coords_to_distance(df):
    objects = infrastruct['type'].unique().tolist()
    for i in range(len(objects)):
        tag = infrastruct[infrastruct['type'] == objects[i]].reset_index()
        # преобразуем координаты в массивы
        flats_coords = [(lat, lon) for lat, lon in zip(df["lat"], df["lon"])]
        tag_coords = [(lat, lon) for lat, lon in zip(tag["lat"], tag["lon"])]
        dist_matrix = distance_matrix(flats_coords, tag_coords)
        nearest_idx = dist_matrix.argmin(axis=1)
        df["lat_" + objects[i]] = tag.loc[nearest_idx, "lat"].values #!impotrant
        df["lon_" + objects[i]] = tag.loc[nearest_idx, "lon"].values
    return df

In [ ]:
coords_to_distance(df)

Функция `coords_to_distance` выполняет те же действия, что и `objects_coords`, но добавляет в датафрейм не координаты объектов, а расстояния от каждой квартиры до ближайшего объекта каждого типа.

In [ ]:
import math

Рассчитаем расстояния от каждой квартиры до ближайшего объекта каждого типа инфраструктуры. 

In [ ]:
def calculate_distance(df):
    objects = infrastruct['type'].unique().tolist()
    for i in range(len(objects)):
        # Радиус Земли в километрах
        R = 6371
        
        # Расчет разницы между широтами и долготами квартир и объектов данного типа в радианах
        df['d_lat'] = (df['lat_' + objects[i]] - df['lat']).apply(math.radians)
        df['d_lon'] = (df['lon_' + objects[i]] - df['lon']).apply(math.radians)
        
        # Расчет параметра a для формулы расстояния между точками на сфере
        df['a'] = ((df['d_lat'] / 2).apply(math.sin))**2 + (df['lat'].apply(math.radians)).apply(math.cos) * (df['lat_' + objects[i]].apply(math.radians)).apply(math.cos) * ((df['d_lon'] / 2).apply(math.sin))**2 
        
        # Расчет параметров y и x для формулы расстояния между точками на сфере
        df['y'] = df['a'].apply(math.sqrt)
        df['x'] = (1 - df['a']).apply(math.sqrt)
        
        # Расчет угла между точками на сфере
        arr = [2 * math.atan2(math.sqrt(df['a'][i]), math.sqrt(1 - df['a'][i])) for i in range(len(df))]
        new_column = pd.Series(arr)
        df = df.assign(angle=new_column)
        
        # Расчет расстояния между квартирами и объектами данного типа в метрах
        df['distance_' + objects[i]] = R * df['angle'] * 1000 
        
        #Удаление врменных столбцов
        df = df.drop(['d_lat', 'd_lon', 'a', 'y', 'x', 'angle'], axis=1)
    return df

Функция использует данные о координатах квартир и объектов инфраструктуры, а также формулу расстояния между точками на сфере для определения расстояний в метрах. Результатом работы функции является дополненный исходный датафрейм с новыми столбцами, содержащими расстояния до каждого типа объектов инфраструктуры.


In [ ]:
df = calculate_distance(df)

Теперь можем удалить все переменные, связанные с географическим положением объекта, поскольку мы воспоьлзовались информацией и сгенерировали новые признаки на основе этих данных. К географическим переменным относятся: ```postcode```, все переменные с приставкой ```lat``` и ```lon```.

In [ ]:
df.iloc[:, 17:43].columns

In [ ]:
df = df.drop(df.iloc[:, 17:43].columns.tolist(), axis=1)

In [ ]:
df